In [1]:
# Import Dependencies
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import concurrent.futures
import sys
import pandas as pd
import json
import queue
import threading

In [ ]:
def crawl_web(start, success_file_path, error_file_path):
    unseen = set([start])
    seen = set([])

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        while unseen:
            url = unseen.pop()
            seen.add(url)
            executor.submit(crawl_website, url, seen, unseen, success_file_path, error_file_path)

    print(f"Finished crawling {start}")
    return seen 

def crawl_website(url, seen, unseen, success_file_path, error_file_path):
    try:
        new_links = getNewLinks(url, seen, unseen, success_file_path)
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            for link in new_links:
                print(f"Adding: {link}")
                unseen.add(link)  
                executor.submit(crawl_website, link, seen, unseen, success_file_path, error_file_path)
    except:
        print(f"Error crawling {url}")
        with open(error_file_path, 'a') as f:
            f.write(f"{url}\n")

def save_links_to_file(links, file_path):
    with open(file_path, 'a') as f:
        for link in links:
            if not is_duplicate_link(link, file_path):
                f.write(f"{link}\n")
                
def getNewLinks(url, seen, unseen, success_file_path):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [link['href'] for link in soup.find_all('a') if link.has_attr('href') and not any(ext in link['href'] for ext in ['.png', '.pdf', '.jpg', '.jpeg', '~json/', 'javascript:;', 'mailto:', 'webcal:'])]
        links = [urljoin(url, link) for link in links]
        links = [link for link in links if link not in seen and link not in unseen and urlparse(link).netloc.endswith('studentaffairs.jhu.edu')] # Change this to limit domain
        save_links_to_file(links, success_file_path)
        return links
    return []

def is_duplicate_link(link, file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if link.strip() == line.strip():
                return True
    return False

def domain(url1, url2):
    return urlparse(url1).netloc == urlparse(url2).netloc

In [ ]:
# crawled_links = crawl_web('https://studentaffairs.jhu.edu', "links.txt", "error.txt")

In [10]:
# Load crawled links from file into a dataframe (in case of termination)
df = pd.read_csv("../data/studentaffairs-jhu-edu/links.txt", header=None, error_bad_lines=False)

# Rename column
df.rename(columns={0: "Link"}, inplace=True)

# Normalize dataframe (remove any links that are not jhu.edu in base url or no https:// in the link)
df = df[df['Link'].str.contains("https://studentaffairs.jhu.edu")]
df


/var/folders/ds/33bb8_h50z75_0klrf43_bqm0000gn/T/ipykernel_16782/3565076695.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("../data/studentaffairs-jhu-edu/links.txt", header=None, error_bad_lines=False)
b'Skipping line 3123: expected 1 fields, saw 2\nSkipping line 8375: expected 1 fields, saw 2\n'


,Link
0,https://studentaffairs.jhu.edu/orientation/new...
1,https://studentaffairs.jhu.edu/orientation/new...
2,https://studentaffairs.jhu.edu/student-success...
3,https://studentaffairs.jhu.edu/student-success...
4,https://studentaffairs.jhu.edu/leed#content
...,...
11775,https://studentaffairs.jhu.edu/fellowships/201...
11776,https://studentaffairs.jhu.edu/fellowships/tag...
11777,https://studentaffairs.jhu.edu/fellowships/tag...
11780,https://studentaffairs.jhu.edu/fellowships/cat...


In [ ]:
def clean_text(text):
    # Remove leading/trailing whitespace, condense multiple whitespaces to single
    return ' '.join(text.split())

def scrape_page(url, q):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'li', 'div', 'table', 'th', 'td', 'tr', 'ol', 'ul', 'blockquote', 'pre', 'code', 'caption', 'dt', 'dd']

    text_data = []
    for tag in tags:
        elements = soup.find_all(tag)
        for element in elements:
            text = clean_text(element.get_text())
            if text:
                text_data.append(text)

    json_data = []
    for i in range(len(text_data) - 1):
        json_data.append({
            'prompt': text_data[i],
            'completion': text_data[i + 1]
        })
        
    # add to queue
    q.put(json_data)
    
def write_to_file(q):
    with open('../data/studentaffairs-jhu-edu/data.json', 'a') as f:
        while True:
            data = q.get()
            if data is None:
                break
            json.dump(data, f)
            f.write('\n')
            q.task_done()

In [21]:
# List of URLs to scrape
urls = df['Link'].tolist()

# create a queue
q = queue.Queue()

# create a separate thread to write data to file
file_writer = threading.Thread(target=write_to_file, args=(q,))
file_writer.start()

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(scrape_page, url, q) for url in urls}
    concurrent.futures.wait(futures)

# stop the file writing thread
q.put(None)
file_writer.join()


In [18]:
# Load pairs into a dataframe
with open('../data/studentaffairs-jhu-edu/data.json', 'r') as f:
    data = []
    for line in f:
        # Error handling
        try:
            data.extend(json.loads(line))
        except:
            pass
        
pc_pairs_df = pd.DataFrame(data)

print(f"Number of Prompt/Completion pairs before cleaning: {len(pc_pairs_df)}")

# Remove any prompts/completion pairs that contains characters less than 15
pc_pairs_df = pc_pairs_df[pc_pairs_df['prompt'].str.len() > 15]
pc_pairs_df = pc_pairs_df[pc_pairs_df['completion'].str.len() > 15]

# # Remove any prompts/completion pairs that are more than 1024 characters
# pc_pairs_df = pc_pairs_df[pc_pairs_df['prompt'].str.len() < 1024]
# pc_pairs_df = pc_pairs_df[pc_pairs_df['completion'].str.len() < 1024]

# Remove duplicates
pc_pairs_df.drop_duplicates(subset='prompt', keep='last', inplace=True)

# Save to JSON file in the format of {"prompt": "prompt text", "completion": "completion text"}
pc_pairs_df.to_json('../data/studentaffairs-jhu-edu/prompt-completion-pairs.json', orient='records', lines=True)

print(f"Number of Prompt/Completion pairs after cleaning: {len(pc_pairs_df)}")

pc_pairs_df

Number of Prompt/Completion pairs before cleaning: 2258820
Number of Prompt/Completion pairs after cleaning: 48863


,prompt,completion
99901,Dr. Charles Lu currently serves as the Associa...,"Dr. Lu has been a clinical professor, teaching..."
99902,"Dr. Lu has been a clinical professor, teaching...",Committed to how research and scholarship conn...
99903,Committed to how research and scholarship conn...,"Over the years, Dr. Lu has served as an educat..."
99904,"Over the years, Dr. Lu has served as an educat...","© Johns Hopkins UniversityBaltimore, Maryland ..."
99917,Office of the Dean of Student Life > Center fo...,Office of the Dean of Student Life
...,...,...
2258814,Choosing majors and courses Covering the costs...,Current Season 2023-2024 Season Auditions Conc...
2258815,Current Season 2023-2024 Season Auditions Conc...,2023-2024 Season Auditions Concerto Competitio...
2258816,2023-2024 Season Auditions Concerto Competitio...,Mission & Advisory Board Staff & Musicians Ens...
2258818,Donate Volunteer,Mission & Advisory Board Staff & Musicians Ens...


In [12]:
# Get dataset statistics
# Amount of links (from links.txt)
print(f"Number of links: {len(df)}")

# Amount of words in dataset
print(f"Number of words in dataset: {pc_pairs_df['prompt'].apply(lambda x: len(x.split())).sum()}")



Number of links: 9404
Number of words in dataset: 1095624
Number of Prompt/Completion pairs after cleaning: 35426
